# Phase 1.5: 토픽 결과 심층 분석

Phase 1에서 학습한 BERTopic 결과물을 활용해 게시판별 토픽 규모, 활성 시간대, 대표 글, 스타일 힌트 등을 시각화/요약합니다.

## 목표
- 각 게시판별 토픽 분포(막대 그래프) 및 활성 시간대 히트맵 생성
- BERTopic 시각화(네트워크/계층/바 차트) HTML로 저장
- 토픽별 키워드, 대표 글, 스타일 통계가 포함된 요약 테이블 생성
- 추후 Phase 2/3에서 참고할 수 있도록 CSV/Markdown로 내보내기


---
## 0. 패키지 설치 (필요 시)

- 이 노트북은 시각화/분석용 패키지만 사용합니다.
- 이미 설치되어 있다면 건너뛰셔도 됩니다. Colab 런타임을 새로 열었다면 한 번만 실행해 주세요.



In [ ]:
%pip install -q pandas plotly seaborn matplotlib bertopic

print("✅ 분석/시각화 패키지 설치 완료")



In [ ]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


---
## 0. 전제 조건 및 입력 데이터
- `phase1_topic_modeling.ipynb`를 먼저 실행하여 `outputs/<게시판>_topics.parquet` 및 관련 파일이 생성되어 있어야 합니다.
- 이 노트북은 해당 출력물을 읽어서 시각화/요약을 수행합니다.


In [ ]:
from pathlib import Path
from typing import Optional
import pandas as pd
import json
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt
from bertopic import BERTopic

pio.renderers.default = "notebook_connected"
sns.set_theme(style="whitegrid")

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
if not OUTPUT_DIR.exists():
    raise FileNotFoundError("outputs 폴더가 없습니다. Phase 1을 먼저 실행하세요.")

print(f"📁 OUTPUT_DIR: {OUTPUT_DIR}")


---
## 1. 산출물 로딩 유틸리티


In [ ]:
def load_topic_artifacts(board_name: str):
    topics_path = OUTPUT_DIR / f"{board_name}_topics.parquet"
    info_path = OUTPUT_DIR / f"{board_name}_topic_info.csv"
    prompt_path = OUTPUT_DIR / f"{board_name}_topics_for_prompt.json"
    model_dir = OUTPUT_DIR / f"bertopic_{board_name}"

    if not topics_path.exists():
        print(f"⚠️ {board_name}: {topics_path} 없음 → 건너뜀")
        return None

    df_topics = pd.read_parquet(topics_path)
    topic_info = pd.read_csv(info_path) if info_path.exists() else None
    prompt_meta = None
    if prompt_path.exists():
        with open(prompt_path, "r", encoding="utf-8") as f:
            prompt_meta = json.load(f)

    return {
        "df": df_topics,
        "topic_info": topic_info,
        "prompt_meta": prompt_meta,
        "model_dir": model_dir if model_dir.exists() else None,
    }

BOARD_ANALYSIS_TARGETS = [
    {"board_name": "익게2", "label": "익게2"},
    {"board_name": "자유게시판", "label": "자유게시판"},
    {"board_name": "연애상담소", "label": "연애상담소"},
    {"board_name": "익게1", "label": "익게1"},
    {"board_name": "인기글", "label": "인기글"},
]

print("분석 대상:", [b["label"] for b in BOARD_ANALYSIS_TARGETS])


---
## 2. 시각화 함수 (토픽 분포 / 시간대 히트맵 / BERTopic HTML)


In [ ]:
def export_plotly(fig, html_path: Path, png_scale: int = 2):
    html_path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(str(html_path))
    try:
        fig.write_image(str(html_path.with_suffix(".png")), scale=png_scale)
    except Exception as exc:
        print(f"⚠️ PNG export 건너뜀({html_path.stem}): {exc}")
    return html_path


def plot_topic_size_chart(df_topics: pd.DataFrame, board_name: str, pretty_name: str, top_n: int = 15):
    valid = df_topics[df_topics["topic"] != -1].copy()
    if valid.empty:
        print(f"⚠️ {pretty_name}: 할당된 토픽이 없습니다.")
        return None

    stats = (
        valid.groupby("topic")
        .agg(doc_count=("id", "count"), avg_prob=("topic_prob", "mean"))
        .reset_index()
        .sort_values("doc_count", ascending=False)
        .head(top_n)
    )
    stats["avg_prob"] = stats["avg_prob"].round(4)

    fig = px.bar(
        stats,
        x="doc_count",
        y=stats["topic"].astype(str),
        text="doc_count",
        orientation="h",
        title=f"[{pretty_name}] Top {top_n} 토픽 크기",
        color="avg_prob",
        color_continuous_scale="Blues",
        labels={"doc_count": "문서 수", "avg_prob": "평균 확률"},
    )
    fig.update_layout(yaxis_title="Topic ID", xaxis_title="문서 수")
    export_path = OUTPUT_DIR / f"{board_name}_topic_bar.html"
    export_plotly(fig, export_path)
    return stats


def plot_time_heatmap(df_topics: pd.DataFrame, board_name: str, pretty_name: str,
                      kind: str = "hour", top_topics: int = 15):
    dt = pd.to_datetime(df_topics["date"], errors="coerce")
    frame = df_topics.copy()
    frame["topic"] = frame["topic"].astype("Int64")
    frame = frame[(frame["topic"] != -1) & dt.notna()].copy()
    if frame.empty:
        return None

    frame["dt"] = dt.loc[frame.index]
    if kind == "hour":
        frame["bucket"] = frame["dt"].dt.hour
        buckets = list(range(24))
        title_suffix = "시간대"
    else:
        frame["bucket"] = frame["dt"].dt.dayofweek
        buckets = list(range(7))
        title_suffix = "요일"

    pivot = frame.pivot_table(index="topic", columns="bucket", values="id", aggfunc="count", fill_value=0)
    if pivot.empty:
        return None

    top_idx = frame["topic"].value_counts().head(top_topics).index
    pivot = pivot.loc[pivot.index.isin(top_idx)]
    pivot = pivot.reindex(columns=buckets, fill_value=0)
    pivot["total"] = pivot.sum(axis=1)
    pivot = pivot.sort_values("total", ascending=False).drop(columns="total")

    fig = px.imshow(
        pivot,
        aspect="auto",
        color_continuous_scale="YlGnBu",
        title=f"[{pretty_name}] 토픽별 {title_suffix}별 활성도",
        labels={"x": "시간대" if kind == "hour" else "요일(0=월)", "y": "Topic", "color": "문서 수"},
    )
    suffix = "hour" if kind == "hour" else "dow"
    export_plotly(fig, OUTPUT_DIR / f"{board_name}_{suffix}_heatmap.html")
    return pivot


def save_markdown_table(df: pd.DataFrame, path: Path, max_rows: int = 200):
    subset = df.head(max_rows)
    if subset.empty:
        return None
    header = "| " + " | ".join(subset.columns) + " |\n"
    divider = "| " + " | ".join(["---"] * len(subset.columns)) + " |\n"
    rows = "\n".join(
        "| " + " | ".join(str(value) for value in row) + " |"
        for row in subset.to_numpy()
    )
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(header + divider + rows)
    return path


In [ ]:
def export_bertopic_visuals(model_dir: Optional[Path], board_name: str):
    if model_dir is None or not model_dir.exists():
        return
    try:
        model = BERTopic.load(str(model_dir))
    except Exception as exc:
        print(f"⚠️ {board_name}: BERTopic 모델 로드 실패 → {exc}")
        return

    viz_dir = OUTPUT_DIR / f"{board_name}_viz"
    viz_dir.mkdir(parents=True, exist_ok=True)

    try:
        model.visualize_topics().write_html(str(viz_dir / "topics.html"))
        model.visualize_hierarchy().write_html(str(viz_dir / "hierarchy.html"))
        model.visualize_barchart().write_html(str(viz_dir / "barchart.html"))
        print(f"✅ {board_name}: BERTopic 시각화 저장 → {viz_dir}")
    except Exception as exc:
        print(f"⚠️ {board_name}: 시각화 생성 실패 → {exc}")


---
## 3. 게시판 루프 실행


In [ ]:
topic_bar_records = []
time_rows = []
summary_rows = []

for cfg in BOARD_ANALYSIS_TARGETS:
    board = cfg["board_name"]
    pretty = cfg["label"]
    artifacts = load_topic_artifacts(board)
    if artifacts is None:
        continue

    df_topics = artifacts["df"]
    stats = plot_topic_size_chart(df_topics, board, pretty, top_n=15)
    if stats is None or stats.empty:
        continue

    stats = stats.assign(board=pretty)
    topic_bar_records.append(stats)

    best_hours, best_dows = time_dow_helper(df_topics)
    time_rows.extend([
        {
            "board": pretty,
            "topic": int(t),
            "best_hours": ", ".join(map(str, best_hours.get(t, []))),
            "best_dows": ", ".join(map(str, best_dows.get(t, []))),
        }
        for t in stats["topic"].tolist()
    ])

    plot_time_heatmap(df_topics, board, pretty, kind="hour")
    plot_time_heatmap(df_topics, board, pretty, kind="dow")

    prompt_meta = artifacts.get("prompt_meta") or []
    prompt_map = {}
    if isinstance(prompt_meta, list):
        for entry in prompt_meta:
            topic_id = entry.get("topic")
            if topic_id is not None:
                prompt_map[int(topic_id)] = entry
    elif isinstance(prompt_meta, dict):
        for key, entry in prompt_meta.items():
            try:
                topic_id = int(key)
            except Exception:
                continue
            prompt_map[topic_id] = entry

    reps_df = representatives(df_topics, k=3)
    reps_map = reps_df.set_index("topic")["representatives"].to_dict() if not reps_df.empty else {}

    for _, row in stats.iterrows():
        topic_id = int(row["topic"])
        prompt_entry = prompt_map.get(topic_id, {})
        keywords = prompt_entry.get("keywords", [])
        style_hints = prompt_entry.get("style_hints", [])

        summary_rows.append({
            "board": pretty,
            "topic": topic_id,
            "doc_count": int(row["doc_count"]),
            "avg_topic_prob": float(row["avg_prob"]),
            "keywords": ", ".join(keywords),
            "best_hours": ", ".join(map(str, best_hours.get(topic_id, []))),
            "best_dows": ", ".join(map(str, best_dows.get(topic_id, []))),
            "representative_titles": " | ".join(rep.get("title", "") for rep in reps_map.get(topic_id, [])) if reps_map else "",
            "style_hints": " | ".join(style_hints) if style_hints else "",
        })

    export_bertopic_visuals(artifacts.get("model_dir"), board)


if topic_bar_records:
    topic_bar_df = pd.concat(topic_bar_records, ignore_index=True)
    topic_bar_csv = OUTPUT_DIR / "topic_bar_stats.csv"
    topic_bar_df.to_csv(topic_bar_csv, index=False)
    print(f"📊 토픽 분포 요약 저장: {topic_bar_csv}")

if time_rows:
    time_df = pd.DataFrame(time_rows).drop_duplicates()
    time_csv = OUTPUT_DIR / "topic_time_summary.csv"
    time_df.to_csv(time_csv, index=False)
    print(f"⏰ 시간대/요일 요약 저장: {time_csv}")

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = OUTPUT_DIR / "topic_summary.csv"
    summary_md = OUTPUT_DIR / "topic_summary.md"
    summary_df.to_csv(summary_csv, index=False)
    save_markdown_table(summary_df, summary_md)
    display(summary_df.head(20))
    print(f"📝 토픽 Summary 저장: {summary_csv}, {summary_md}")
else:
    print("⚠️ 요약 데이터가 없어 저장을 건너뜁니다.")


---
## 4. 완료 안내
- 위 산출물은 모두 `outputs/` 디렉터리에 저장됩니다.
- Phase 2에서는 `topic_summary.csv`와 `인기글_topics_for_prompt.json` 등을 활용해 Instruction Tuning 데이터셋을 구성합니다.
- 추가 분석이 필요하면 본 노트북을 수정해 확장할 수 있습니다.

⚡ **Tip**: 시각화 HTML/PNG 파일은 Google Drive에서 직접 열어 확인하세요.
